In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42

In [17]:
workload = 'Google_RPC_read'
# workload = 'Facebook_Hadoop'
note = '_incLineRates' # Should start with _
forUtilExp = False
forStarTopo = True
num_hosts = 64
load = 0.8
simIdx = 0
TOR_BW = 400e9  # bps
SRC_PORT_NO_START = 1000
DST_PORT_NO_START = 2000
HPCC_TRACE_START_TIME = 2.0 # seconds

if (forUtilExp):
  trace_path = 'outputs/bolt-utilization/'
elif (forStarTopo):
  trace_path = 'outputs/bolt-star-topo/'
else:
  trace_path = 'outputs/bolt-' + str(num_hosts) + 'host-fattree/'

trace_path += workload + '_load-' + str(int(load*100)) + 'p' 

if (forStarTopo or forUtilExp):
  trace_path += '_nHost-' + str(num_hosts)

trace_path += '_' + str(simIdx)

# if (forUtilExp):
#   trace_path += '_MSGAGG'

if TOR_BW == 100e9 and not forUtilExp:
  rate_info = '_100G'
elif TOR_BW == 200e9 and not forUtilExp:
  rate_info = '_200G'
elif TOR_BW == 400e9 and not forUtilExp:
  rate_info = '_400G'
else:
  rate_info = ''

bolt_trace_filename = trace_path + '_DEFAULT' + rate_info + note + '.tr'
if (forUtilExp):
  hpcc_trace_filename = trace_path[:-11] + '_star_nHost-' + str(num_hosts) + '.tr'
elif (forStarTopo):
  hpcc_trace_filename = trace_path[:-11] + '_star_nHost-' + str(num_hosts) + rate_info + '.tr'
else:
  hpcc_trace_filename = trace_path[:-2] + '_fat_nHost-' + str(num_hosts) + '.tr'

print(bolt_trace_filename)
print(hpcc_trace_filename)

outputs/bolt-star-topo/Google_RPC_read_load-80p_nHost-64_0_DEFAULT_400G_incLineRates.tr
outputs/bolt-star-topo/Google_RPC_read_load-80p_star_nHost-64_400G.tr


Example line from the bolt trace:

> "+ 1000000095 48609 10.0.25.1:1025 10.0.20.1:1020 0"



In [18]:
hpcc_trace_str = ''
num_msgs = 0
with open(bolt_trace_filename,'r') as f:
    for line in f:
      msgLog = line.split()
      if (msgLog[0] != '+'):
        continue

      time = float(msgLog[1])*1e-9        # in seconds
      while (time < HPCC_TRACE_START_TIME):
        time += 1.0

      src_node = int(msgLog[3].split(':')[1]) - SRC_PORT_NO_START 
      dst_node = int(msgLog[4].split(':')[1]) - SRC_PORT_NO_START 

      msgSize = int(msgLog[2])            # in bytes

      hpcc_trace_str += str(src_node) + ' ' + str(dst_node) + ' 3 100 ' + str(msgSize) + ' ' + str(time) + '\n'
      num_msgs += 1

In [19]:
with open(hpcc_trace_filename, 'w') as f:
    f.write(str(num_msgs) + '\n' + hpcc_trace_str)